In [ ]:
from google.colab import drive
import os
drive.mount('/gdrive')
os.getcwd()
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Mounted at /gdrive


'/content'

In [ ]:
root = "/gdrive/My Drive/CS470/cs470_music_generator"
root_data = '/gdrive/My Drive/CS470/data'
root_midi = '/gdrive/My Drive/CS470/data/midi'
os.chdir(root)
os.getcwd()

'/gdrive/My Drive/CS470/cs470_music_generator'

In [ ]:
!git clone https://github.com/bearpelican/musicautobot.git

fatal: destination path 'musicautobot' already exists and is not an empty directory.


In [ ]:
os.chdir('musicautobot')
os.getcwd()
!apt-get update
!apt install musescore fluidsynth
!pip install torch fastai music21 pebble fluidsynth midi2audio
from musicautobot.numpy_encode import *
from musicautobot.config import *
from musicautobot.music_transformer import *
from musicautobot.multitask_transformer import *
from musicautobot.utils import midifile
os.chdir('..')
os.getcwd()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,370 kB]
Get:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-securit

'/gdrive/My Drive/CS470/cs470_music_generator'

In [ ]:
with open("data/music_vocab.pkl", "rb") as fd:
  music_vocab = pickle.load(fd)

In [ ]:
import os
files = os.listdir("./data/midi_eval")

In [ ]:
with open("data/dataset_np.pkl", "rb") as fd:
  music_data = pickle.load(fd)
beatles = list(map(lambda x: x["np"], music_data))
beatles[0]

tensor([  0,   1,   8,  ..., 189,  49, 189])

In [ ]:
others = []
for f in files:
  item = MusicItem.from_file("./data/midi_eval/" + f, music_vocab)
  others.append(item.to_tensor())

In [ ]:
with open("data/music_eval_vocab.pkl", "wb") as fd:
  pickle.dump(music_vocab, fd)

In [ ]:
len(beatles), len(others)

(288, 194)

In [ ]:
chunk_len_music = 400

In [ ]:
x = list(map(lambda x: x.numpy().tolist(), beatles)) + list(map(lambda x: x.numpy().tolist(), others))
y = ["beatles"] * len(beatles) + ["others"] * len(others)
len(x), len(y)

(482, 482)

In [ ]:
with open("data/midi_eval.pkl", "wb") as fd:
  pickle.dump({"x": x, "y": y}, fd)

In [ ]:
all_x = []
all_y = []
for i in range(0, len(x)):
  x_ = x[i]
  y_ = y[i]
  for j in range(0, len(x_) - chunk_len_music, chunk_len_music):
    if (np.random.rand() > 0.3):
      all_x.append(x_[j: j + chunk_len_music])
      all_y.append(y_)

In [ ]:
len(all_x), len(all_y)

(6803, 6803)

In [ ]:
len(all_x[0])

400

In [ ]:
x = all_x
y = all_y

In [ ]:
import random
random.seed(3)

random.shuffle(x)
random.shuffle(y)

x_train, x_test = x[:-500], x[-500:]
y_train, y_test = y[:-500], y[-500:]
len(x_train), len(x_test), len(y_train), len(y_test)

(6303, 500, 6303, 500)

In [ ]:
import collections
collections.Counter(y_test)

Counter({'beatles': 290, 'others': 210})

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(C=1, random_state=0).fit(x_train, y_train)
clf.score(x_test, y_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.516